In [1]:
import numpy as np

import astropy.units as u
from astropy.time import Time
from astropy.coordinates import solar_system_ephemeris

from poliastro.bodies import Sun,Venus, Earth, Jupiter, Saturn
from poliastro.twobody import Orbit
from poliastro.maneuver import Maneuver
from poliastro.iod import izzo
from poliastro.plotting import OrbitPlotter2D
from poliastro.util import norm
import poliastro.twobody.propagation as Propagation

from scipy import optimize as opt
 
solar_system_ephemeris.set("jpl")

<ScienceState solar_system_ephemeris: 'jpl'>

In [2]:
cassini_venus2 = Time("1999-06-24")
cassini_earth2 = Time("1999-08-18")
cassini_jupiter = Time("2000-12-30")
cassini_saturn = Time("2004-07-01")

#cassini transit
cassini_js_time = cassini_saturn-cassini_jupiter
cassini_ej_time = cassini_jupiter-cassini_earth2
cassini_ve_time = cassini_earth2-cassini_venus2

In [3]:
tmod=[0,0,0,0]
plot_on=0;

def trajectory_calculator(t=[0,0,0,0],plot_on=0,disp_on=0):
    TIME_SCALE="tdb"
    
    #start with cassini style assumptions
    cassini_venus2 = Time("1999-06-24")
    cassini_earth2 = Time("1999-08-18")
    cassini_jupiter = Time("2000-12-30")
    cassini_saturn = Time("2004-07-01")

    #cassini transit
    cassini_js_time = cassini_saturn-cassini_jupiter
    cassini_ej_time = cassini_jupiter-cassini_earth2
    cassini_ve_time = cassini_earth2-cassini_venus2


    tmod_s  = t[0] #arrival time in saturn relative to 2020
    tmod_js = t[1] #transit times
    tmod_ej = t[2] 
    tmod_ve = t[3]


    time_js = tmod_js*u.year
    time_ej = tmod_ej*u.year
    time_ve = tmod_ve*u.year

    date_arrive_saturn = Time("2020-01-01", scale=TIME_SCALE)+tmod_s*u.year;


    #create target orbit
    o_sf = Orbit.from_body_ephem(Saturn, date_arrive_saturn)
    r_sf, v_sf = o_sf.rv()
    #o_sf.plot()
    if disp_on:
        print('computed saturn')

    ################## Jupiter ################## 

    #guess flyby date of Jupiter
    date_flyby_jupiter = date_arrive_saturn-time_js

    #construct j orbit
    o_j = Orbit.from_body_ephem(Jupiter, date_flyby_jupiter)
    #o_j.plot()

    #compute transfer lambert trajectory
    (v_jo, v_si), = izzo.lambert(Sun.k, o_j.r, o_sf.r, time_js)
    trx_js = Orbit.from_vectors(Sun, o_j.r, v_jo, epoch=date_flyby_jupiter)
    #trx_js.plot()
    if disp_on:
        print('computed J-S')
    ################## Earth ################## 
    #guess flyby date of Earth
    date_flyby_earth = date_arrive_saturn-time_js-time_ej

    #construct j orbit
    o_e2 = Orbit.from_body_ephem(Earth, date_flyby_earth)
    #o_e2.plot()

    #compute transfer lambert trajectory
    (v_eo, v_ji), = izzo.lambert(Sun.k, o_e2.r, o_j.r, time_ej)
    trx_ej = Orbit.from_vectors(Sun, o_e2.r, v_eo, epoch=date_flyby_earth)
    #trx_ej.plot()
    if disp_on:
        print('computed E-J')                            

    ################## Venus2 ################## 
    #guess flyby date of Venus2
    date_flyby_venus2 = date_arrive_saturn-time_js-time_ej-time_ve

    #construct j orbit
    o_v2 = Orbit.from_body_ephem(Venus, date_flyby_venus2)
    #o_v2.plot()

    #compute transfer lambert trajectory
    (v_v2o, v_ei), = izzo.lambert(Sun.k, o_v2.r, o_e2.r, time_ve)
    trx_v2e = Orbit.from_vectors(Sun, o_v2.r, v_v2o, epoch=date_flyby_venus2)
    #trx_v2e.plot()    
    if disp_on:
        print('computed V-E')                            

    ################## Sum delta v ##################                             
    delv_e = norm(v_eo-v_ei)
    delv_j = norm(v_jo-v_ji)
    delv_s = norm(v_si-v_sf)

    total_deltav=sum([delv_e,delv_j,delv_s]) 

    if disp_on:
        print('Total delta-v: ', total_deltav)

    ################## Plot ##################  

    if plot_on:
        op = OrbitPlotter2D()

        op.plot(o_v2,label="Venus2 Orbit")
        op.plot(o_e2,label="Earth2 Orbit")
        op.plot(o_j, label="Jupiter Orbit")
        op.plot(o_sf, label="Saturn Orbit")


        op.plot(trx_v2e, label="V2-E")
        op.plot(trx_ej, label="E-J")
        op.plot(trx_js, label="J-S")
        
    orbits = (o_v2,o_e2,o_j,o_sf)
    trajectories = (trx_v2e,trx_ej,trx_js)
    deltavs = (delv_e,delv_j,delv_s)
    times = (date_flyby_venus2,date_flyby_earth,date_flyby_jupiter,date_arrive_saturn)
    return (total_deltav,orbits,trajectories,deltavs,times)

def print_times(times):
    print("Venus:   ",times[0])
    print("Earth:   ",times[1])
    print("Jupiter: ",times[2])
    print("Saturn:  ",times[3])
    print("**")
    print("V-E: ", (times[1]-times[0]).to(u.year))
    print("E-J: ", (times[2]-times[1]).to(u.year))
    print("E-S: ", (times[3]-times[2]).to(u.year))
        



In [4]:
"""
tmod_s = 0

#date_arrive_saturn = Time("2020-01-01", scale="utc")+tmod_s*u.year;
#print(date_arrive_saturn)  # produces 2020-01-01 00:00:00.000
date_arrive_saturn = Time("2020-01-01", scale="tdb")+tmod_s*u.year;
print(date_arrive_saturn)  # produces 2020-01-01 00:00:00.000

del tmod_s
del date_arrive_saturn
"""

'\ntmod_s = 0\n\n#date_arrive_saturn = Time("2020-01-01", scale="utc")+tmod_s*u.year;\n#print(date_arrive_saturn)  # produces 2020-01-01 00:00:00.000\ndate_arrive_saturn = Time("2020-01-01", scale="tdb")+tmod_s*u.year;\nprint(date_arrive_saturn)  # produces 2020-01-01 00:00:00.000\n\ndel tmod_s\ndel date_arrive_saturn\n'

In [5]:
cassini_js_time.to(u.year).value

3.5017111567419574

In [6]:
t_guess_cassini = [
    27,
    cassini_js_time.to(u.year).value,
    cassini_ej_time.to(u.year).value,
    cassini_ve_time.to(u.year).value]

In [7]:
sol=trajectory_calculator(t=t_guess_cassini,plot_on=0,disp_on=1)

computed saturn
computed J-S
computed E-J
computed V-E
Total delta-v:  82.09823281566113 km / s


In [8]:
sol

(<Quantity 82.09823282 km / s>,
 (1 x 1 AU x 24.4 deg (ICRS) orbit around Sun (☉) at epoch 2041-12-23 18:00:00.000 (TDB),
  1 x 1 AU x 23.4 deg (ICRS) orbit around Sun (☉) at epoch 2042-02-16 18:00:00.000 (TDB),
  5 x 5 AU x 23.2 deg (ICRS) orbit around Sun (☉) at epoch 2043-07-01 18:00:00.000 (TDB),
  9 x 10 AU x 22.6 deg (ICRS) orbit around Sun (☉) at epoch 2046-12-31 18:00:00.000 (TDB)),
 (30292872 x -2883253670 km x 24.0 deg (HCRS) orbit around Sun (☉) at epoch 2041-12-23 18:00:00.000 (TDB),
  1 x 10 AU x 23.2 deg (HCRS) orbit around Sun (☉) at epoch 2042-02-16 18:00:00.000 (TDB),
  0 x 47 AU x 27.1 deg (HCRS) orbit around Sun (☉) at epoch 2043-07-01 18:00:00.000 (TDB)),
 (<Quantity 38.07630083 km / s>,
  <Quantity 29.27968946 km / s>,
  <Quantity 14.74224253 km / s>),
 (<Time object: scale='tdb' format='iso' value=2041-12-23 18:00:00.000>,
  <Time object: scale='tdb' format='iso' value=2042-02-16 18:00:00.000>,
  <Time object: scale='tdb' format='iso' value=2043-07-01 18:00:00.000

In [9]:
t_guess_cassini

[27, 3.5017111567419574, 1.3689253935660506, 0.15058179329226556]

In [10]:
t_guess_cassini = [18,cassini_js_time.to(u.year).value,cassini_ej_time.to(u.year).value,cassini_ve_time.to(u.year).value]
# bounds on input vector, so min/max transit times for each leg: Sat Arrive, JS, EJ, VE, 
bounds = ((10,20),(0.01,5),(0.01,3),(0.01,1))
options = {'maxiter': 30,'disp': True}

def total_deltav_calculator(tmod):
    print(tmod)
    sol=trajectory_calculator(tmod,plot_on=0,disp_on=0)
    print(sol[0])
    return sol[0].to(u.km/u.s).value

opt_sol = opt.minimize(total_deltav_calculator,x0=t_guess_cassini,options=options,bounds=bounds)

opt_sol

[18.          3.50171116  1.36892539  0.15058179]
79.18286022601885 km / s
[18.00000001  3.50171116  1.36892539  0.15058179]
79.18285874723891 km / s
[18.          3.50171117  1.36892539  0.15058179]
79.18286160893722 km / s
[18.          3.50171116  1.3689254   0.15058179]
79.18286142992953 km / s
[18.          3.50171116  1.36892539  0.1505818 ]
79.18285916236258 km / s
[2.e+01 1.e-02 1.e-02 1.e+00]
5616.601107936685 km / s
[2.e+01 1.e-02 1.e-02 1.e+00]
5616.601052797132 km / s
[2.000000e+01 1.000001e-02 1.000000e-02 1.000000e+00]
5616.599434074284 km / s
[2.000000e+01 1.000000e-02 1.000001e-02 1.000000e+00]
5616.597249285565 km / s
[2.00000000e+01 1.00000000e-02 1.00000000e-02 1.00000001e+00]
5616.601107357299 km / s
[18.74169344  2.20682154  0.86497237  0.46558575]
90.91153624783321 km / s
[18.74169345  2.20682154  0.86497237  0.46558575]
90.91153741615192 km / s
[18.74169344  2.20682155  0.86497237  0.46558575]
90.91153491007086 km / s
[18.74169344  2.20682154  0.86497238  0.46558

59.81314470320543 km / s
[18.81250776  2.67937328  1.41643454  0.32327658]
59.81314588155011 km / s
[18.81250775  2.67937329  1.41643454  0.32327658]
59.813143392013075 km / s
[18.81250775  2.67937328  1.41643455  0.32327658]
59.813143328564614 km / s
[18.81250775  2.67937328  1.41643454  0.32327659]
59.81314519976459 km / s
[18.7735178   2.70493773  1.40271186  0.18361702]
41.51783775294312 km / s
[18.77351781  2.70493773  1.40271186  0.18361702]
41.51783915121774 km / s
[18.7735178   2.70493774  1.40271186  0.18361702]
41.51783622307685 km / s
[18.7735178   2.70493773  1.40271187  0.18361702]
41.51783616863422 km / s
[18.7735178   2.70493773  1.40271186  0.18361703]
41.51783888230959 km / s
[18.76903306  2.70787823  1.40113343  0.16755296]
39.699846804608306 km / s
[18.76903307  2.70787823  1.40113343  0.16755296]
39.69984734125792 km / s
[18.76903306  2.70787824  1.40113343  0.16755296]
39.69984613651638 km / s
[18.76903306  2.70787823  1.40113344  0.16755296]
39.699846117608686 km 

27.14947330045603 km / s
[20.          3.34863319  1.98308695  0.17860471]
27.1494738289176 km / s
[20.          3.43179361  1.89914145  0.17137496]
26.94761653107287 km / s
[20.00000001  3.43179361  1.89914145  0.17137496]
26.947616453502206 km / s
[20.          3.43179362  1.89914145  0.17137496]
26.94761653435537 km / s
[20.          3.43179361  1.89914146  0.17137496]
26.947616530953624 km / s
[20.          3.43179361  1.89914145  0.17137497]
26.947616336950354 km / s
[20.          3.40289691  1.92756002  0.17306602]
26.929837351263487 km / s
[20.00000001  3.40289691  1.92756002  0.17306602]
26.929837339468467 km / s
[20.          3.40289692  1.92756002  0.17306602]
26.929837287916804 km / s
[20.          3.40289691  1.92756003  0.17306602]
26.929837285733857 km / s
[20.          3.40289691  1.92756002  0.17306603]
26.92983741232578 km / s


      fun: 26.929837351263487
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([-1.17950201, -6.33466826, -6.55296297,  6.10622948])
  message: b'STOP: TOTAL NO. of ITERATIONS REACHED LIMIT'
     nfev: 255
      nit: 30
   status: 1
  success: False
        x: array([20.        ,  3.40289691,  1.92756002,  0.17306602])

In [11]:
t_guess_cassini

[18, 3.5017111567419574, 1.3689253935660506, 0.15058179329226556]

In [12]:
opt_sol

      fun: 26.929837351263487
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([-1.17950201, -6.33466826, -6.55296297,  6.10622948])
  message: b'STOP: TOTAL NO. of ITERATIONS REACHED LIMIT'
     nfev: 255
      nit: 30
   status: 1
  success: False
        x: array([20.        ,  3.40289691,  1.92756002,  0.17306602])

In [13]:
sol=trajectory_calculator(t=opt_sol.x,plot_on=0,disp_on=0)

print("Tmod: ",opt_sol.x)
print()
print("Total deltav: ", sol[0])
print("Delta vs: ",[x.value for x in sol[3]])
print()

print('Times:')
print_times(sol[4])
print()
t_ve=sol[4][1]-sol[4][0]
t_ej=sol[4][2]-sol[4][1]
t_js=sol[4][3]-sol[4][2]


trx_ve = sol[2][0]
trx_ej = sol[2][1]
trx_js = sol[2][2]


trx_ve_half = trx_ve.propagate(t_ve/2)
trx_ej_half = trx_ej.propagate(t_ej/2)
trx_js_half = trx_js.propagate(t_ej/2)


op=OrbitPlotter2D()
op.plot(sol[1][0],label='V')
op.plot(sol[1][1],label='E')
op.plot(sol[1][2],label='J')
op.plot(sol[1][3],label='S')
op.plot(trx_ve_half,label='VE')
op.plot(trx_ej_half,label='EJ')
op.plot(trx_js_half,label='JS')

Tmod:  [20.          3.40289691  1.92756002  0.17306602]

Total deltav:  26.929837351263487 km / s
Delta vs:  [1.4801750574031514, 14.89569911459923, 10.553963179261107]

Times:
Venus:    2034-06-30 20:07:04.282
Earth:    2034-09-02 01:12:52.517
Jupiter:  2036-08-06 02:12:20.453
Saturn:   2040-01-01 00:00:00.000
**
V-E:  0.17306602007602725 yr
E-J:  1.927560015195392 yr
E-S:  3.402896910634561 yr



/usr/local/lib/python3.6/dist-packages/poliastro/twobody/propagation.py:230: UserWarning:

Frame <class 'astropy.coordinates.builtin_frames.icrs.ICRS'> does not support 'obstime', time values were not returned



FigureWidget({
    'data': [{'hoverinfo': 'none',
              'line': {'color': 'rgb(31, 119, 180)', 'dash':…

In [14]:
norm(sol[1][2].v).to(u.km/u.s)

<Quantity 13.43495202 km / s>

In [15]:
#tmod = opt_sol.x
tmod = [27.238877,4.798,1.2879,0.1313]
sol=trajectory_calculator(t=tmod,plot_on=0,disp_on=0)

#print("Tmod: ",opt_sol.x)
print()
print("Total deltav: ", sol[0])
print("Delta vs: ",[x.value for x in sol[3]])
print()

print('Times:')
print_times(sol[4])
print()
t_ve=sol[4][1]-sol[4][0]
t_ej=sol[4][2]-sol[4][1]
t_js=sol[4][3]-sol[4][2]

o_v = sol[1][0]  # orbit of venus at venus flyby
o_e = sol[1][1]  # orbit of earth at earth flyby
o_j = sol[1][2]  # orbit of jupiter at jupiter flyby
o_s = sol[1][3]  # orbit of saturn at saturn flyby/arrive
trx_ve = sol[2][0]
trx_ej = sol[2][1]
trx_js = sol[2][2]


trx_ve_half = trx_ve.propagate(t_ve/2)
trx_ej_half = trx_ej.propagate(t_ej/2)
trx_js_half = trx_js.propagate(t_js/2)


op=OrbitPlotter2D()
op.plot(sol[1][0],label='V')
op.plot(sol[1][1],label='E')
op.plot(sol[1][2],label='J')
op.plot(sol[1][3],label='S')
op.plot(trx_ve_half,label='VE')
op.plot(trx_ej_half,label='EJ')
op.plot(trx_js_half,label='JS')


Total deltav:  6.010065391517322 km / s
Delta vs:  [0.1916879332331373, 0.07762259806409005, 5.740754860220094]

Times:
Venus:    2041-01-08 04:01:14.095
Earth:    2041-02-25 02:59:46.975
Jupiter:  2042-06-10 12:43:40.015
Saturn:   2047-03-28 23:59:44.815
**
V-E:  0.1313 yr
E-J:  1.2879 yr
E-S:  4.798 yr



FigureWidget({
    'data': [{'hoverinfo': 'none',
              'line': {'color': 'rgb(31, 119, 180)', 'dash':…

In [16]:
trx_ve.p.to(u.AU)

<Quantity 1.30253528 AU>

In [17]:
norm(trx_ve.r).to(u.AU)

<Quantity 0.72505093 AU>

In [18]:
trx_ve.p

<Quantity 1.94856504e+08 km>

In [19]:
trx_ve.ecc

<Quantity 0.86513652>

In [20]:
trx_ve.inc.to(u.deg)

<Quantity 22.82693059 deg>

In [21]:
trx_ve.raan.to(u.deg)

<Quantity 0.66526436 deg>

In [22]:
trx_ve.argp.to(u.deg)

<Quantity 87.40301688 deg>

In [23]:
trx_ve.nu.to(u.deg)

<Quantity 337.01896162 deg>

In [24]:
trx_ve.r

<Quantity [45778973.34576497, 90711406.76156612, 37955442.62063536] km>

In [25]:
trx_ve.v

<Quantity [-46.22297003,  10.79180186,   4.76801816] km / s>

In [26]:
trx_ve_half = trx_ve.propagate(t_ve/2)

In [27]:
trx_ve_half.plot()

FigureWidget({
    'data': [{'hoverinfo': 'none',
              'line': {'color': 'rgb(31, 119, 180)', 'dash':…

In [28]:
norm(o_e.v.to(u.km/u.s))

<Quantity 30.08181407 km / s>

In [29]:
print_times(sol[4])

Venus:    2041-01-08 04:01
Earth:    2041-02-25 02:59
Jupiter:  2042-06-10 12:43
Saturn:   2047-03-28 23:59
**
V-E:  0.1313 yr
E-J:  1.2879 yr
E-S:  4.798 yr


In [30]:
sol

(<Quantity 6.01006539 km / s>,
 (1 x 1 AU x 24.5 deg (ICRS) orbit around Sun (☉) at epoch 2041-01-08 04:01 (TDB),
  1 x 1 AU x 23.4 deg (ICRS) orbit around Sun (☉) at epoch 2041-02-25 02:59 (TDB),
  5 x 5 AU x 23.2 deg (ICRS) orbit around Sun (☉) at epoch 2042-06-10 12:43 (TDB),
  9 x 10 AU x 22.6 deg (ICRS) orbit around Sun (☉) at epoch 2047-03-28 23:59 (TDB)),
 (1 x 10 AU x 22.8 deg (HCRS) orbit around Sun (☉) at epoch 2041-01-08 04:01 (TDB),
  1 x 10 AU x 22.6 deg (HCRS) orbit around Sun (☉) at epoch 2041-02-25 02:59 (TDB),
  1 x 10 AU x 22.4 deg (HCRS) orbit around Sun (☉) at epoch 2042-06-10 12:43 (TDB)),
 (<Quantity 0.19168793 km / s>,
  <Quantity 0.0776226 km / s>,
  <Quantity 5.74075486 km / s>),
 (<Time object: scale='tdb' format='iso' value=2041-01-08 04:01>,
  <Time object: scale='tdb' format='iso' value=2041-02-25 02:59>,
  <Time object: scale='tdb' format='iso' value=2042-06-10 12:43>,
  <Time object: scale='tdb' format='iso' value=2047-03-28 23:59>))

In [31]:
(Time("2047-03-29", scale="utc")-Time("2020-01-01", scale="tdb")).to(u.year)

<Quantity 27.23887967 yr>

In [32]:
(norm(sol[1][3].v).to(u.km/u.s))**2

<Quantity 83.66078139 km2 / s2>

In [33]:
norm(sol[1][1].v).to(u.km/u.s)

<Quantity 30.08181407 km / s>

In [34]:
Earth.R

<<class 'astropy.constants.constant.Constant'> name='Earth equatorial radius' value=6378136.6 uncertainty=0.0001 unit='m' reference='IAU Working Group on Cartographic Coordinates and Rotational Elements: 2009'>

In [35]:
v_inf = 3.5*u.km/u.s;
qmin = 10*6000*u.km;
2*v_inf*(1+((v_inf)**2*qmin/(Earth.k)).to(u.m/u.m))**-1

<Quantity 2.4613638 km / s>

In [36]:
t_guess_cassini = [-15.5,cassini_js_time.to(u.year).value,cassini_ej_time.to(u.year).value,cassini_ve_time.to(u.year).value]

t_guess_online_traj_tool = [20, 7.93, 2.19,0.15]
t_guess_online_traj_tool = [12, 2, 2,0.15]


t_guess = opt_sol.x

t_guess=t_guess_cassini

#t_guess=t_guess_online_traj_tool

bounds = ((-20,-10),(0.01,10),(0.01,3),(0.01,1))

options = {'maxiter': 60,'disp': True}

def calc_max_deltav(v_inf,qratio=10, body=Earth):
    return 2*v_inf*(1+((v_inf)**2*(qratio*body.R)/(body.k)).to(u.m/u.m))**-1

def weighted_deltav_calculator(tmod):
        print(tmod)
        sol = trajectory_calculator(tmod, plot_on=0, disp_on=0)
        print(sol[3])
        
        # 10 is a weight
        # 10*V_inf at Saturn + sum of all other deltaVs (including saturn)
        return 10*sol[3][2].to(u.km/u.s).value + sum(sol[3]).to(u.km/u.s).value

def saturn_deltav_calculator(tmod):
    print(tmod)
    sol=trajectory_calculator(tmod,plot_on=0,disp_on=0)
    print(sol[3])
    
    
    
    return 5*sol[3][2].to(u.km/u.s).value + sum(sol[3]).to(u.km/u.s).value


def earthflybyConstraint(ts):
    
    sol = trajectory_calculator(t=ts,plot_on=0,disp_on=0)
    
    deltavearth = sol[3][0].to(u.km/u.s)
    print('deltav about earth')
    print(deltavearth)
    v_earth = sol[1][1].v.to(u.km/u.s)
    #print(norm(v_earth))
    v_sat_leaving_earth = sol[2][1].v
    #print(v_sat_leaving_earth)
    #print(norm(v_sat_leaving_earth))
    v_inf = norm(v_sat_leaving_earth-v_earth).to(u.km/u.s)
    #print('v_inf')
    #print(v_inf)
    print('max deltav')
    max_deltav = calc_max_deltav(v_inf,qratio=2, body=Earth)
    print(max_deltav)
    
    return float((max_deltav-deltavearth)/max_deltav)


def jupiterflybyConstraint(ts):
    
    sol = trajectory_calculator(t=ts,plot_on=0,disp_on=0)
    
    deltavjup = sol[3][1].to(u.km/u.s)
    print('deltav about jupiter')
    print(deltavjup)
    v_jup = sol[1][2].v.to(u.km/u.s)
    #print(norm(v_earth))
    v_sat_leaving_jup = sol[2][2].v
    #print(v_sat_leaving_earth)
    #print(norm(v_sat_leaving_earth))
    v_inf = norm(v_sat_leaving_jup-v_jup).to(u.km/u.s)
    #print('v_inf')
    #print(v_inf)
    print('max deltav')
    max_deltav = calc_max_deltav(v_inf,qratio=2, body=Jupiter)
    print(max_deltav)
    
    return float((max_deltav-deltavjup)/max_deltav)


earth_constraint ={'type':'ineq','fun':earthflybyConstraint}
jupiter_constraint ={'type':'ineq','fun':jupiterflybyConstraint}

opt_sol = opt.minimize(weighted_deltav_calculator,
                       x0=t_guess,
                       options=options,
                       bounds=bounds,
                       constraints=(earth_constraint,jupiter_constraint))

opt_sol

deltav about earth
5.637717307592528 km / s
max deltav
3.4600425812061193 km / s
deltav about jupiter
2.0019264191480715 km / s
max deltav
18.63197974958147 km / s
[-15.5          3.50171116   1.36892539   0.15058179]
(<Quantity 5.63771731 km / s>, <Quantity 2.00192642 km / s>, <Quantity 5.36526106 km / s>)
deltav about earth
5.637717307592528 km / s
max deltav
3.4600425812061193 km / s
deltav about jupiter
2.0019264191480715 km / s
max deltav
18.63197974958147 km / s
[-15.5          3.50171116   1.36892539   0.15058179]
(<Quantity 5.63771731 km / s>, <Quantity 2.00192642 km / s>, <Quantity 5.36526106 km / s>)
[-15.49999999   3.50171116   1.36892539   0.15058179]
(<Quantity 5.6377163 km / s>, <Quantity 2.00192662 km / s>, <Quantity 5.36526108 km / s>)
[-15.5          3.50171117   1.36892539   0.15058179]
(<Quantity 5.63771831 km / s>, <Quantity 2.00192622 km / s>, <Quantity 5.36526102 km / s>)
[-15.5          3.50171116   1.36892541   0.15058179]
(<Quantity 5.63771827 km / s>, <Quantit

deltav about jupiter
3.2308514111563427 km / s
max deltav
16.53909426841718 km / s
deltav about jupiter
3.230851295673537 km / s
max deltav
16.539094123334554 km / s
deltav about jupiter
3.230851193120818 km / s
max deltav
16.539094207163902 km / s
deltav about jupiter
3.230851345546779 km / s
max deltav
16.539094207163902 km / s
[-1.40227945e+01  5.13948585e+00  1.17013971e+00  1.00000000e-02]
(<Quantity 792.17023176 km / s>, <Quantity 5.32891367 km / s>, <Quantity 3.9184949 km / s>)
deltav about earth
792.1702317599998 km / s
max deltav
2.863135152464194 km / s
deltav about jupiter
5.32891367431375 km / s
max deltav
15.278824270869796 km / s
[-14.39659478   4.65592811   1.30562935   0.11117089]
(<Quantity 4.55830548 km / s>, <Quantity 3.39445767 km / s>, <Quantity 4.28760925 km / s>)
deltav about earth
4.558305477011091 km / s
max deltav
3.2586398139619854 km / s
deltav about jupiter
3.394457669409614 km / s
max deltav
16.40815865913762 km / s
[-14.39659478   4.65592811   1.30562935 

(<Quantity 4.40735533 km / s>, <Quantity 4.64929254 km / s>, <Quantity 3.88481137 km / s>)
[-12.40293019   6.55383506   1.40277742   0.11948844]
(<Quantity 4.407355 km / s>, <Quantity 4.64929269 km / s>, <Quantity 3.88481137 km / s>)
deltav about earth
4.4073548654818815 km / s
max deltav
3.319195433249265 km / s
deltav about earth
4.407354374850709 km / s
max deltav
3.3191952291311235 km / s
deltav about earth
4.4073553561154855 km / s
max deltav
3.3191956373674265 km / s
deltav about earth
4.407355327452775 km / s
max deltav
3.319195643551365 km / s
deltav about earth
4.407355000806199 km / s
max deltav
3.319195433249265 km / s
deltav about jupiter
4.649292694547548 km / s
max deltav
14.245076069964478 km / s
deltav about jupiter
4.6492927963621495 km / s
max deltav
14.245076122075119 km / s
deltav about jupiter
4.649292604530288 km / s
max deltav
14.245075999618567 km / s
deltav about jupiter
4.64929254143248 km / s
max deltav
14.245076069964478 km / s
deltav about jupiter
4.6492926

(<Quantity 4.08124773 km / s>, <Quantity 5.60419894 km / s>, <Quantity 4.13613269 km / s>)
[-11.15339343   7.48834847   1.71248042   0.1147784 ]
(<Quantity 4.08124773 km / s>, <Quantity 5.60419899 km / s>, <Quantity 4.13613271 km / s>)
[-11.15339343   7.48834847   1.71248041   0.11477841]
(<Quantity 4.08124749 km / s>, <Quantity 5.60419915 km / s>, <Quantity 4.13613271 km / s>)
deltav about earth
4.081247490432437 km / s
max deltav
3.374404273804567 km / s
deltav about earth
4.081247254271913 km / s
max deltav
3.3744040753433437 km / s
deltav about earth
4.081247726595469 km / s
max deltav
3.3744044722657947 km / s
deltav about earth
4.0812477340230995 km / s
max deltav
3.374404478570539 km / s
deltav about earth
4.081247492754311 km / s
max deltav
3.374404273804567 km / s
deltav about jupiter
5.60419915258547 km / s
max deltav
14.162294105591196 km / s
deltav about jupiter
5.604199369585316 km / s
max deltav
14.162294153703355 km / s
deltav about jupiter
5.604198940268546 km / s
max d

deltav about earth
4.082484187239412 km / s
max deltav
3.3754368459039994 km / s
deltav about earth
4.082484668009118 km / s
max deltav
3.375437242788977 km / s
deltav about earth
4.082484675714831 km / s
max deltav
3.375437249099866 km / s
deltav about earth
4.082484427553863 km / s
max deltav
3.3754370443464876 km / s
deltav about jupiter
5.605089607244903 km / s
max deltav
14.210537053758104 km / s
deltav about jupiter
5.605089825062321 km / s
max deltav
14.210537101935525 km / s
deltav about jupiter
5.60508939403466 km / s
max deltav
14.210536987468426 km / s
deltav about jupiter
5.605089449606744 km / s
max deltav
14.210537053758104 km / s
deltav about jupiter
5.605089607244903 km / s
max deltav
14.210537053758104 km / s
[-12.40770947   6.23273797   1.71378102   0.11474182]
(<Quantity 4.08199868 km / s>, <Quantity 5.26471096 km / s>, <Quantity 4.37438245 km / s>)
deltav about earth
4.0819986774962915 km / s
max deltav
3.3750364704262004 km / s
deltav about jupiter
5.26471096166919

...

deltav about jupiter
5.576791960287851 km / s
max deltav
14.299743472646917 km / s
deltav about jupiter
5.576792179237079 km / s
max deltav
14.299743521005148 km / s
deltav about jupiter
5.576791745917656 km / s
max deltav
14.299743406098798 km / s
deltav about jupiter
5.576791802861685 km / s
max deltav
14.299743472646917 km / s
deltav about jupiter
5.576791960287851 km / s
max deltav
14.299743472646917 km / s
[-11.26058433   7.37957998   1.71405633   0.11474085]
(<Quantity 4.08201307 km / s>, <Quantity 5.576792 km / s>, <Quantity 4.15512747 km / s>)
deltav about earth
4.082013069767534 km / s
max deltav
3.375048964993918 km / s
deltav about jupiter
5.5767919981618155 km / s
max deltav
14.299743479258373 km / s
[-11.26058433   7.37957998   1.71405633   0.11474085]
(<Quantity 4.08201307 km / s>, <Quantity 5.576792 km / s>, <Quantity 4.15512747 km / s>)
[-11.26058431   7.37957998   1.71405633   0.11474085]
(<Quantity 4.08201283 km / s>, <Quantity 5.57679222 km / s>, <Quantity 4.15512749

(<Quantity 4.08181905 km / s>, <Quantity 5.57688421 km / s>, <Quantity 4.15512452 km / s>)
deltav about earth
4.0818190545417 km / s
max deltav
3.374887988270428 km / s
deltav about jupiter
5.5768842143009225 km / s
max deltav
14.299753580412222 km / s
[-11.26058708   7.37957771   1.71405461   0.11474088]
(<Quantity 4.08199247 km / s>, <Quantity 5.57680222 km / s>, <Quantity 4.15512725 km / s>)
deltav about earth
4.081992465778619 km / s
max deltav
3.375031927616408 km / s
deltav about jupiter
5.5768022248517255 km / s
max deltav
14.29974466468415 km / s
[-11.26058542   7.37957905   1.71405561   0.11474094]
(<Quantity 4.08200378 km / s>, <Quantity 5.5767969 km / s>, <Quantity 4.15512742 km / s>)
deltav about earth
4.082003775857293 km / s
max deltav
3.3750412814838815 km / s
deltav about jupiter
5.576796897228014 km / s
max deltav
14.299744085310099 km / s
[-11.26058542   7.37957905   1.71405561   0.11474094]
(<Quantity 4.08200378 km / s>, <Quantity 5.5767969 km / s>, <Quantity 4.15512

deltav about jupiter
5.576758404509279 km / s
max deltav
14.299761763759644 km / s
[-11.26059742   7.37956914   1.71405352   0.11474079]
(<Quantity 4.08200265 km / s>, <Quantity 5.57678525 km / s>, <Quantity 4.1551265 km / s>)
deltav about earth
4.08200265137999 km / s
max deltav
3.375040350065711 km / s
deltav about jupiter
5.576785254241489 km / s
max deltav
14.299749404447782 km / s
[-11.26058942   7.37957575   1.71405491   0.11474089]
(<Quantity 4.08200339 km / s>, <Quantity 5.576793 km / s>, <Quantity 4.15512711 km / s>)
deltav about earth
4.082003393754164 km / s
max deltav
3.3750409649966997 km / s
deltav about jupiter
5.576793001531431 km / s
max deltav
14.299745838259783 km / s
[-11.26058703   7.37957772   1.71405532   0.11474092]
(<Quantity 4.08200361 km / s>, <Quantity 5.57679531 km / s>, <Quantity 4.15512729 km / s>)
deltav about earth
4.0820036148761645 km / s
max deltav
3.3750411481584304 km / s
deltav about jupiter
5.57679530911738 km / s
max deltav
14.29974477604504 km 

(<Quantity 4.08199192 km / s>, <Quantity 5.57677517 km / s>, <Quantity 4.15511608 km / s>)
[-11.26057869   7.37959333   1.71404756   0.11473838]
(<Quantity 4.08199167 km / s>, <Quantity 5.57677533 km / s>, <Quantity 4.15511608 km / s>)
deltav about earth
4.08199167165676 km / s
max deltav
3.37503127048384 km / s
deltav about earth
4.0819914316229795 km / s
max deltav
3.3750310720314225 km / s
deltav about earth
4.081991911693073 km / s
max deltav
3.3750314689362746 km / s
deltav about earth
4.081991919304136 km / s
max deltav
3.375031475244538 km / s
deltav about earth
4.081991671662676 km / s
max deltav
3.37503127048384 km / s
deltav about jupiter
5.576775325837917 km / s
max deltav
14.29970213565551 km / s
deltav about jupiter
5.576775544787436 km / s
max deltav
14.299702184013752 km / s
deltav about jupiter
5.576775111467569 km / s
max deltav
14.299702069107424 km / s
deltav about jupiter
5.576775168409732 km / s
max deltav
14.29970213565551 km / s
deltav about jupiter
5.57677532583

     fun: 55.36504387982364
     jac: array([ 1.17829061e+01, -1.32311783e+01,  6.05450821e+00,  3.97682190e-04])
 message: 'Iteration limit exceeded'
    nfev: 642
     nit: 61
    njev: 60
  status: 9
 success: False
       x: array([-11.26057869,   7.37959333,   1.71404756,   0.11473836])

In [37]:
calc_max_deltav(3.5*u.km/u.s)

<Quantity 2.36473485 km / s>

In [38]:
#sol = trajectory_calculator(t=opt_sol.x,plot_on=0,disp_on=1)
sol = trajectory_calculator(t=t_guess_cassini,plot_on=0,disp_on=1)

computed saturn
computed J-S
computed E-J
computed V-E
Total delta-v:  13.004904787450446 km / s


In [39]:
opt_sol.x

array([-11.26057869,   7.37959333,   1.71404756,   0.11473836])

In [40]:
calc_max_deltav(9*u.km/u.s)

<Quantity 1.2892989 km / s>

In [41]:
sol = trajectory_calculator(t=opt_sol.x,plot_on=0,disp_on=1)

print(opt_sol)

#sol = trajectory_calculator([ 21.82450945,   9.33104326,   2.39418937,   0.2])

#print("Tmod: ",opt_sol.x)
print()
print("Total deltav: ", sol[0])
print("Delta vs: ",[x.value for x in sol[3]])
print()

print('Times:')
print_times(sol[4])
print()
t_ve=sol[4][1]-sol[4][0]
t_ej=sol[4][2]-sol[4][1]
t_js=sol[4][3]-sol[4][2]


trx_ve = sol[2][0]
trx_ej = sol[2][1]
trx_js = sol[2][2]


trx_ve_half = trx_ve.propagate(t_ve/2)
trx_ej_half = trx_ej.propagate(t_ej/2)
trx_js_half = trx_js.propagate(t_js/2)


op=OrbitPlotter2D()
op.plot(sol[1][0],label='V')
op.plot(sol[1][1],label='E')
op.plot(sol[1][2],label='J')
op.plot(sol[1][3],label='S')
op.plot(trx_ve_half,label='VE')
op.plot(trx_ej_half,label='EJ')
op.plot(trx_js_half,label='JS')

computed saturn
computed J-S
computed E-J
computed V-E
Total delta-v:  13.81388307770643 km / s
     fun: 55.36504387982364
     jac: array([ 1.17829061e+01, -1.32311783e+01,  6.05450821e+00,  3.97682190e-04])
 message: 'Iteration limit exceeded'
    nfev: 642
     nit: 61
    njev: 60
  status: 9
 success: False
       x: array([-11.26057869,   7.37959333,   1.71404756,   0.11473836])

Total deltav:  13.81388307770643 km / s
Delta vs:  [4.081991671656882, 5.576775325837827, 4.155116080211721]

Times:
Venus:    1999-07-13 17:06:52.706
Earth:    1999-08-24 14:54:40.049
Jupiter:  2001-05-11 16:15:07.263
Saturn:   2008-09-27 01:46:01.795
**
V-E:  0.11473836231848278 yr
E-J:  1.7140475579353476 yr
E-S:  7.379593331956557 yr



FigureWidget({
    'data': [{'hoverinfo': 'none',
              'line': {'color': 'rgb(31, 119, 180)', 'dash':…

In [42]:
trx_js_final = trx_js.propagate(t_js)

In [43]:
trx_js_final.v

<Quantity [-0.14691288, -6.13349699, -2.63858195] km / s>

In [44]:
o_s=sol[1][3]

In [45]:
o_s.v.to(u.km/u.s)

<Quantity [-3.40679344, -8.5939558 , -3.40298418] km / s>

In [46]:
norm(trx_js_final.v-o_s.v)

<Quantity 4.15511608 km / s>

In [47]:
(Saturn.k/(4*Saturn.R))**0.5

<Quantity 12543.67952454 m / s>

In [48]:
print()
print("Total deltav: ", sol[0])
print("Delta vs: ",[x.value for x in sol[3]])
print()



Total deltav:  13.81388307770643 km / s
Delta vs:  [4.081991671656882, 5.576775325837827, 4.155116080211721]



In [49]:
earthflybyConstraint(t_guess_cassini)
print('jup')
jupiterflybyConstraint(t_guess_cassini)

deltav about earth
5.637717307592528 km / s
max deltav
3.4600425812061193 km / s
jup
deltav about jupiter
2.0019264191480715 km / s
max deltav
18.63197974958147 km / s


0.8925542832240874